# `01` Import Necessary Libraries

In [ ]:
import torch
import cv2
from PIL import Image
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

c:\Users\FreeComp\anaconda3\Lib\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\FreeComp/.cache\torch\hub\master.zip


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\FreeComp\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5  2025-4-7 Python-3.12.7 torch-2.6.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:03<00:00, 3.77MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# `02` Read The image

In [37]:
path = 'person_bottle.jpg'
img = Image.open(path)

results = model(img)

detections = results.pandas().xyxy[0]  

C:\Users\FreeComp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


# `03` Split the detections

In [38]:
# Filter detections for persons, bottles, and cups
persons = detections[detections['name'] == 'person']
bottles = detections[detections['name'] == 'bottle']
cups = detections[detections['name'] == 'cup']

# `04` Holding algorithm 

In [39]:
def holding(person, item, threshold=0.5):
    # Calculate the center of the person and item bounding boxes
    person_center = ((person['xmin'] + person['xmax']) / 2, (person['ymin'] + person['ymax']) / 2)
    item_center = ((item['xmin'] + item['xmax']) / 2, (item['ymin'] + item['ymax']) / 2)

    # Calculate the distance between the centers
    distance = ((person_center[0] - item_center[0]) ** 2 + (person_center[1] - item_center[1]) ** 2) ** 0.5

    # Calculate the size of the person bounding box
    person_size = ((person['xmax'] - person['xmin']) ** 2 + (person['ymax'] - person['ymin']) ** 2) ** 0.5

    # Determine if the item is in a certain distance using threshold of the person size
    return distance < threshold * person_size




# `05` Applying the algorithm on the Photo

In [40]:
# Flags
bottle_held = False
cup_held = False


for _, person in persons.iterrows():
    # this block is to check if any person is holding a bottle
    for _, bottle in bottles.iterrows():
        if holding(person, bottle):
            print("Person is holding a bottle")
            bottle_held = True
    # this block is to check if any person is holding a cup
    for _, cup in cups.iterrows():
        if holding(person, cup):
            print("Person is holding a cup")
            cup_held = True

# Check if no person is holding bottle or cup
if not bottle_held and not cup_held :
    print("No person is holding a bottle or cup")


Person is holding a bottle


# `06` Save the photo if the person hold bottle or cup 

In [ ]:
if bottle_held or cup_held:
    results.render()  

    annotated_img = results.ims[0]  

    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR)

    cv2.imwrite(f'annotated {path}', annotated_img)

yes


# `07` Apply the algorithm in videos

In [ ]:
cap = cv2.VideoCapture('video.mp4')

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter('annotated_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
flag_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    detections = results.pandas().xyxy[0]

    persons = detections[detections['name'] == 'person']
    cups = detections[detections['name'] == 'cup']
    bottles = detections[detections['name'] == 'bottle']

    item_held = False
    for _, person in persons.iterrows():
        # Check if any person is holding a cup
        for _, cup in cups.iterrows():
            if holding(person, cup):
                item_held = True
                flag_list.append(1)
                break
        for _, bottle in bottles.iterrows():
           # Check if any person is holding a bottle
            if holding(person, bottle):
                item_held = True
                flag_list.append(2)
                break
        if item_held:
            break


    if not item_held:
        flag_list.append(0)
    
    # Save the annotaded video
    results.render()  
    annotated_frame = results.ims[0]  

    annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

    out.write(annotated_frame)

cap.release()
out.release()

C:\Users\FreeComp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\FreeComp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\FreeComp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\FreeComp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\FreeComp/.cache\torch\hub\ultralytics_yolov

Tell the class of the video


In [47]:
if 1 in flag_list:
    print("The person hold a cup")
elif 2 in flag_list:
    print("The person hold a bottle")
else:
    print("The person didn't hold a bottle or a cup")

The person hold a cup
